In [1]:
# Quelle: https://pytorch.org/tutorials/beginner/text_sentiment_ngrams_tutorial.html
# pip install torchtext
# pip install spacy
# python -m spacy download de
# pip install torch


import pandas as pd

df=pd.read_csv("Amazon-Deutsch-Dataset.csv")
df.head()

,Unnamed: 0,web-scraper-order,web-scraper-start-url,author,title,date,content,rating,next,next-href,Unnamed: 9
0,0,1582056286-2631,https://www.amazon.de/Apple-iPhone-11-128-GB-S...,MHS,Das Beste iPhone aller Zeiten,5. Januar 2020,Ich bin sehr zufrieden mit dem iPhone 11. Der ...,"5,0 von 5 Sternen",Weiter→,https://www.amazon.de/Apple-iPhone-11-128-GB-S...,NaN
1,1,1582056184-2351,https://www.amazon.de/Apple-iPhone-11-128-GB-S...,Amazon Customer,besser als beim hersteller,21. September 2019,"gestern bestellt, heute geliefert. besser geht...","5,0 von 5 Sternen",NaN,NaN,NaN
2,2,1582056243-2561,https://www.amazon.de/Apple-iPhone-11-128-GB-S...,Joko Müller,Gutes Handy mit kleinen Schwächen,27. Oktober 2019,"Ich mach es mal kurz:\nGut: Optik, Verarbeitun...","4,0 von 5 Sternen",Weiter→,https://www.amazon.de/Apple-iPhone-11-128-GB-S...,NaN
3,3,1582056201-2410,https://www.amazon.de/Apple-iPhone-11-128-GB-S...,Davorko,Ein sehr edles Stück dieses IPHONE 11,2. Januar 2020,Amazon hat wieder super-schnell geliefert. Dan...,"5,0 von 5 Sternen",Weiter→,https://www.amazon.de/Apple-iPhone-11-128-GB-S...,NaN
4,4,1582056246-2585,https://www.amazon.de/Apple-iPhone-11-128-GB-S...,Chiara Natalia Sozzi,Super,18. Oktober 2019,"Viel früher angekommen als angegeben, tolles H...","5,0 von 5 Sternen",Weiter→,https://www.amazon.de/Apple-iPhone-11-128-GB-S...,NaN


In [2]:
# uns interessiert erstmal nur content und rating als Zahl
df = df[["content", "rating"]]
df.head()

,content,rating
0,Ich bin sehr zufrieden mit dem iPhone 11. Der ...,"5,0 von 5 Sternen"
1,"gestern bestellt, heute geliefert. besser geht...","5,0 von 5 Sternen"
2,"Ich mach es mal kurz:\nGut: Optik, Verarbeitun...","4,0 von 5 Sternen"
3,Amazon hat wieder super-schnell geliefert. Dan...,"5,0 von 5 Sternen"
4,"Viel früher angekommen als angegeben, tolles H...","5,0 von 5 Sternen"


In [3]:
df.rating= df.rating.str[0]
df.head()

,content,rating
0,Ich bin sehr zufrieden mit dem iPhone 11. Der ...,5
1,"gestern bestellt, heute geliefert. besser geht...",5
2,"Ich mach es mal kurz:\nGut: Optik, Verarbeitun...",4
3,Amazon hat wieder super-schnell geliefert. Dan...,5
4,"Viel früher angekommen als angegeben, tolles H...",5


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3413 entries, 0 to 3412
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   content  3409 non-null   object
 1   rating   3409 non-null   object
dtypes: object(2)
memory usage: 53.5+ KB


In [5]:
# remove rows with missing values
df = df.dropna()

In [6]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

/home/codespace/.python/current/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
tokenizer = get_tokenizer('spacy', language='de_core_news_sm')

# create iterator from tokenized df
def df_iterator_content(df):
    for _, row in df.iterrows():
        yield tokenizer(row['content'])

In [8]:
vocab = build_vocab_from_iterator(df_iterator_content(df), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

In [9]:
vocab(['Hallo', 'Welt', 'Foo', 'Iphone'])

[1029, 3019, 0, 806]

In [10]:
def text_pipeline(x):                           # die benutzen wir später um Text in Vektoren zu übersetzen
    return vocab(tokenizer(x))   
label_pipeline = lambda x: int(x) - 1           # (andere Schreibweise) die ratings liegen als Text vor und gehen von 1 bis 5. Daher -1

In [11]:
text_pipeline('Die letzte Bestellung war fehlerhaft') # Beispiel

[32, 1131, 1026, 36, 8268]

In [12]:
for _, label in df_iterator(df):
    print(label_pipeline(label))

NameError: name 'df_iterator' is not defined

In [ ]:
from torch.utils.data import DataLoader
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def collate_batch(batch):
    '''
    Extrahiert aus einer Batch von Text die Labels und die Texte und übersetzt diese in Vektoren mittels text_pipeline und label_pipeline
    '''
    label_list, text_list, offsets = [], [], [0]
    for (_label, _text) in batch:
         label_list.append(label_pipeline(_label))
         processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
         text_list.append(processed_text)
         offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)


dataloader = DataLoader(df_iterator_content(df), batch_size=8, shuffle=False, collate_fn=collate_batch)

In [ ]:
from torch import nn

class TextClassificationModel(nn.Module):

    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

In [ ]:
def df_iterator(df):
    for _, row in df.iterrows():
        yield tokenizer(row['content']), row['rating']

num_class = len(set([label for (text, label) in df_iterator(df)]))
vocab_size = len(vocab)
emsize = 64
model = TextClassificationModel(vocab_size, emsize, num_class).to(device)

In [ ]:
import time

def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predicted_label = model(text, offsets)
        loss = criterion(predicted_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predicted_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predicted_label = model(text, offsets)
            loss = criterion(predicted_label, label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count

In [ ]:
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset
# Hyperparameters
EPOCHS = 10 # epoch
LR = 5  # learning rate
BATCH_SIZE = 64 # batch size for training

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None
train_iter = df_iterator(df)
train_dataset = to_map_style_dataset(train_iter)
num_train = int(len(train_dataset) * 0.95)
split_train_, split_valid_ = \
    random_split(train_dataset, [num_train, len(train_dataset) - num_train])

train_dataloader = DataLoader(split_train_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)

for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(train_dataloader)
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} '.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_val))
    print('-' * 59)

TypeError: int() argument must be a string, a bytes-like object or a real number, not 'list'

: 